In [1]:
import matplotlib.pyplot as plt

<img src="equations_article.png" width="450px" height="300px">
<img src="equations2_article.png" width="450px" height="300px">

In [ ]:
# Parameters

# defaultclock.dt=0.1*ms # for better precision

thet = {"e": 1.3, "i": 2.0}
b = {"e": 4.0, "i": 3.7}
k = {"e": 0.9945, "i": 0.9994}

# tau doivent etre entre 10 et 20 ms
tau_cx = 10
tau_th = 10
tau_nrt = 10
tau_dcn = 10
tau_gpe = 10
tau_gpi = 10
tau_stn = 10

# Healthy state parameters
w = [0, 20, 5, 8, 25, 15, 5, 19, 5, 15, 20, 20]

w1 = 20
w2 = 5
w3 = 8
w4 = 25
w5 = 15
w6 = 5
w7 = 19
w8 = 5
w9 = 15
w10 = 20
w11 = 20
ext = 3.42

# # Tremor band parameters
# w1 = 20
# w2 = 12
# w3 = 8
# w4 = 9
# w5 = 15
# w6 = 5
# w7 = 5
# w8 = 5
# w9 = 15
# w10 = 20
# w11 = 20
# ext = 3.42

# # Beta band parameters
# w1 = 20
# w2 = 5
# w3 = 8
# w4 = 20
# w5 = 15
# w6 = 5
# w7 = 5
# w8 = 5
# w9 = 15
# w10 = 20
# w11 = 20
# ext = 3.42

In [ ]:
# Z function

def Z(x,p):
    return (1 / (1 + exp(- b[p] * (x - thet[p])))) - (1 / (1 + exp(b[p] * thet[p])))


# Equations

# dE_cx/dt = (- E_cx + (k_e - E_cx) * Z_e(w1 * E_th)) / tau_cx : 1
# dE_th/dt = (- E_th + (k_e - E_th) * Z_e(w2 * E_cx - w3 * I_nrt + w4 * E_dcn - w5 * I_gpi)) / tau_th : 1
# dI_nrt/dt = (- I_nrt + (k_i - I_nrt) * Z_i(w6 * E_cx)) / tau_nrt : 1
# dE_dcn/dt = (- E_dcn + (k_e - E_dcn) * Z_e(ext)) / tau_dcn : 1
# dI_gpe/dt = (- I_gpe + (k_i - I_gpe) * Z_i(w7 * E_stn - w8 * I_gpe)) / tau_gpe : 1
# dI_gpi/dt = (- I_gpi + (k_i - I_gpi) * Z_i(w9 * E_stn)) / tau_gpi : 1
# dE_stn/dt = (- E_stn + (k_e - E_stn) * Z_e(w10 * E_cx - w11 * I_gpe)) / tau_stn : 1